In [1]:
import json
import requests
from src import utils
import time
import os

In [2]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api.json'

In [3]:
api = utils.read_json_file(api_path)

In [4]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_preprocess.jsonl'
# in_path = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-train.jsonl'

In [6]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [7]:
output_file_name = 'zero-shot-cot-public_mwp_data_v2_preprocess'

In [8]:
# os.makedirs(output_path, exist_ok=True)
# output_file_name = 'result-' + input_file.split('/')[-1]
output_file = output_path + '/' + output_file_name
print(f'output_file : {output_file}')

output_file : /userhomes/philhoon/kt-ai-challenge/result/zero-shot-cot-public_mwp_data_v2_preprocess


In [9]:
jsonl_writer = utils.JSONLWriter(output_file)

In [10]:
data = utils.read_jsonlines(input_file)

In [11]:
len(data)

3750

In [12]:
# api

In [13]:
# url = api['url']

In [14]:
# headers = api['headers']

In [15]:
# question = "질문 : 파운드 스털링은 가치가 있나요? 용어설명 :  사실목록 : ['1파운드는 법정 화폐입니다.', '법정 화폐는 정부 법령에 의해 뒷받침되며 내재적 가치가 없습니다', '2020년 5월 기준 1파운드 스털링은 약 1.24달러의 가치가 있습니다.']. 사실목록을 바탕으로 질문에 대한 답변을 사실 또는 거짓으로 대답하세요"/

In [16]:
# headers

In [17]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [18]:
model = model_api(api['url'], api['headers'])

In [19]:
BODY_TEMP = api['body']

In [20]:
# body = json.dumps(body_temp)

In [21]:
# print(body)

In [22]:
# response = api.generate(body)

In [23]:
# context = response.json()['data'][0]['result'][0]

In [24]:
# print(context)

In [26]:
def get_mwp_zs_cot(que):
    prompt = f"질문: {que} 정답: 질문을 단계적으로 풀어보면, "
    return prompt

In [27]:
def get_mwp_zs(que):
    prompt = f'질문: {que} 정답 : '
    return prompt

In [28]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [ ]:
# from pprint import pprint

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [29]:
cnt = 0
split_cnt = 0
for ins in data:

    kor_que = ins['question']
    
    
    prompt = get_mwp_zs_cot(kor_que)


    body = get_body(prompt, BODY_TEMP)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    
    ins['zs_infer'] = res_context
#     jsonl_writer.write_json_line(ins)

    if cnt == 5:
        break
    else:
        cnt += 1
    


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 지영이네 반 친구들이 키가 작은 순서대로 줄을 서있습니다. 지영이는 앞에서부터 6번째에 줄을 섰으며, 지영이네 반은 모두 18명입니다. 키가 큰 사람부터 다시 줄을 세운다면, 지영이는 앞에서부터 몇 번째로 줄을 서게 될까요? 정답: 질문을 단계적으로 풀어보면, 
res_context : 
정답: 1단계: 지영이네 반 친구들의 키가 작은 순서대로 줄을 서있습니다. 지영이는 앞에서부터 6번째에 줄을 섰으며, 지영이네 반은 모두 18명입니다. 2단계: 키가 큰 사람부터 다시 줄을 세운다면, 지영이는 앞에서부터 몇 번째로 줄을 서게 될까요? 6번째 + 18번째 = 36번째 3단계: 지영이는 앞에서부터 몇 번째로 줄을 서게 될까요? 36번째


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 반짇고리 안에는 여러 가지 모양의 단추가 있습니다. 동그란 단추는 6개가 들어있고, 네모난 단추와 세모난 단추는 각각 3개씩 들어 있습니다. 동그란 단추는 네모난 단추보다 몇 개가 더 많습니까? 정답: 질문을 단계적으로 풀어보면, 
res_context : 
1. 동그란 단추는 6개가 들어있고, 네모난 단추는 3개가 들어있습니다. 2. 네모난 단추와 세모난 단추는 각각 3개씩 들어 있습니다. 3. 동그란 단추는 네모난 단추보다 2개가 더 많습니다. 따라서, 동그란 단추는 네모난 단추보다 2개가 더 많습니다.


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

prompt : 
질문: 집게의 길이는 4cm 8mm이고, 빨대의 길이는 3cm 6mm입니다. 집게와 빨대의 길이의 차는 몇 mm일까요? 정답: 질문을 단계적으로 풀어보면, 
res_context : 
1. 집게의 길이는 4cm 8mm이고, 빨대의 길이는 3cm 6mm이므로, 집게와 빨대의 길이의 차는 4cm + 3cm = 12cm입니다. 2. 집게와 빨대의 길이의 차를 계산하기 위해, 집게의 길이를 빨대의 길이로 나누어야 합니다. 집게의 길이 = 4cm 8mm / 3cm = 12cm 3. 집게와 빨대의 길이의 차를 계산하기 위해, 빨대의 길이를 집게의 길이로 나누어야 합니다. 빨대의 길이 = 3cm 6mm / 4cm = 8cm 따라서, 집게와 빨대의 길이의 차는 12cm입니다.


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 연지가 하트모양 초콜릿을 만듭니다. 녹인 초콜릿을 10칸 짜리 틀에 부었더니 7개의 틀을 가득 채웠고, 여덟번째 틀에는 빈 칸이 3개 남았습니다. 연지가 만든 초콜릿의 개수는 전부 몇 개입니까? 정답: 질문을 단계적으로 풀어보면, 
res_context : 
연지가 만든 초콜릿의 개수는 7개입니다. 7개의 틀을 가득 채웠고, 여덟번째 틀에는 빈 칸이 3개 남았습니다. 총 7 + 3 = 11개의 초콜릿이 만들어졌습니다.


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 숲 속 동물들이 100m 달리기 시합을 했더니 호랑이는 15초, 고라니는 20초, 삵은 18초, 늑대는 21초를 기록했습니다. 2번째로 빠른 동물의 기록은 몇 초였나요? 정답: 질문을 단계적으로 풀어보면, 
res_context : 
1. 주어진 시간을 계산합니다. 2. 주어진 시간에서 1을 빼줍니다. 3. 15초 + 20초 = 15 4. 18초 + 21초 = 21 5. 2번째로 빠른 동물의 기록을 계산합니다. 6. 2번째로 빠른 동물의 기록에서 1을 빼줍니다. 7. 15초 + 20초 = 15 8. 따라서, 2번째로 빠른 동물의 기록은 15초였습니다.
